# 🛒 Sistema de Recomendación para E-Commerce
**Prueba Técnica - Lead Data Scientist**  
**Autor:** David Caleb  
**Fecha:** Octubre 2025

---

## 📋 Tabla de Contenidos

1. [Introducción y Objetivos](#seccion-1-introduccion)
2. [Metodología CRISP-DM](#seccion-2-metodologia)
3. [Análisis Exploratorio de Datos (EDA)](#seccion-3-eda)
4. [Preprocesamiento y Preparación](#seccion-4-preprocesamiento)
5. [Sistema de Recomendación](#seccion-5-sistema-recomendacion)
6. [Evaluación de Modelos](#seccion-6-evaluacion)
7. [Sistema de Actualización](#seccion-7-actualizacion)
8. [Monitorización Continua](#seccion-8-monitorizacion)
9. [Framework de A/B Testing](#seccion-9-ab-testing)
10. [Conclusiones y Recomendaciones](#seccion-10-conclusiones)

---

<a id="seccion-1-introduccion"></a>
## 🎯 1. Introducción y Objetivos

### Contexto de Negocio
La empresa de e-commerce busca **optimizar la experiencia del cliente** mediante un sistema de recomendación personalizado que:
- Aumente las conversiones
- Mejore el engagement del usuario
- Incremente el valor promedio del pedido

### Objetivos del Proyecto

**Técnicos:**
- Implementar múltiples algoritmos de recomendación
- Alcanzar Precision@10 mayor a 0.003
- Manejar el problema de cold start
- Gestionar datos dispersos

**De Negocio:**
- Aumentar CTR en recomendaciones en 15%
- Incrementar tasa de conversión en 10%
- Mejorar retención de clientes

### Alcance

**En Scope:** Análisis exploratorio, 5 algoritmos, evaluación offline, plan de deployment

**Out of Scope:** Deep Learning, procesamiento en tiempo real, NLP

---


In [ ]:
!pip install -r requirements.txt

In [ ]:
# Import necessary libraries

# Data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime
import hashlib

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import precision_score, recall_score

# Sparse matrices - scipy
from scipy.sparse import csr_matrix, coo_matrix
from scipy import stats

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Configure visualization styles
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

<a id="seccion-2-metodologia"></a>
# 📊 2. Metodología: CRISP-DM

---

## Framework de Trabajo

Este proyecto sigue la metodología **CRISP-DM** (Cross-Industry Standard Process for Data Mining):

### Fases Aplicadas:
- **Fase 1-2:** Business Understanding & Data Understanding
  - Análisis exploratorio completo (Celdas 3-24)
  - Comprensión de objetivos de negocio
  
- **Fase 3:** Data Preparation
  - Preprocesamiento y limpieza (Celdas 13-24)
  - Creación de features temporales y RFM
  
- **Fase 4:** Modeling
  - Implementación de 5 algoritmos (Celdas 27-40)
  - User-Based CF, Item-Based CF, SVD, Híbrido
  
- **Fase 5:** Evaluation
  - Métricas: Precision@10, Recall@10, F1-Score (Celdas 33-40)
  - Comparación de modelos
  
- **Fase 6:** Deployment
  - Estrategia de producción (Celdas 42-50)
  - Monitorización y A/B Testing


<a id="seccion-3-eda"></a>
# 📊 3. Análisis Exploratorio de Datos (EDA)

---

## Carga de Datasets

En esta sección cargaremos y exploraremos los datos de transacciones y clientes.


In [ ]:
# Load the datasets
data_transactions = pd.read_csv("dataset_sample_1.csv")
data_customers = pd.read_csv("dataset_sample_2.csv")

In [ ]:
data_transactions.head()

In [ ]:
data_customers.head()

# Overview Datasets

In [ ]:
# Check a overview of the dataset
def overview(data):
    print("=".center(50,"="))

    # Print the shape of the dataset to see how many rows and columns it has.
    print(f"\nOverview")
    print(f"Shape: {data.shape}")
    print(f"Memory Usage: {data.memory_usage().sum()/1024/1024:.2f} MB")
    print("=".center(50,"="))

    # Display Index, Columns, and Data Types
    print("Information about the features:")
    print(data.info())
    print("=".center(50,"="))
    dtype_counts = data.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"{str(dtype):<20}: {count} columns")
    print("=".center(50,"="))

    # Display summary statistics
    print("Basic statistics check:")
    print(data.describe())
    print("=".center(50,"="))

    # I always run this part to understand the unique values in each column.
    # It helps me get a sense of the data, especially which features are categorical or have low variability.
    print("Checking the number of unique values:")
    unique_counts = {}
    for column in data.columns:
        unique_counts[column] = data[column].nunique()
    unique_data = pd.DataFrame(unique_counts, index=["Unique Count"]).transpose()
    print(unique_data)
    print("=".center(50, "="))

    # Check for Missing Values
    print("Check for missing values:")
    missing_values = data.isnull().sum()
    missing_pct = (missing_values / len(data)) * 100
    missing_data = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage (%)': missing_pct.round(2)
    })
    print(missing_data[missing_data['Missing Values'] > 0])

## Overview Data Transactiones (dataset_sample1)

In [ ]:
overview(data_transactions)

## Overview Data Customers (dataset_sample2)

In [ ]:
overview(data_customers)

<a id="seccion-4-preprocesamiento"></a>
# 🔧 4. Preprocesamiento y Preparación de Datos

---

## Limpieza y Transformación

Aplicaremos técnicas de limpieza, manejo de valores nulos e ingeniería de características.


In [ ]:
# Limpieza y preprocesamiento mejorado
def preprocess_data(transactions_df, customers_df):
    # Copiar los datos para no modificar los originales
    transactions = transactions_df.copy()
    customers = customers_df.copy()
    
    print("INICIANDO PREPROCESAMIENTO")
    
    # 1. Manejo de valores nulos en transacciones
    print("Manejo de valores nulos en transacciones...")
    transactions['CATEGORIA'] = transactions['CATEGORIA'].fillna('DESCONOCIDA')
    
    # 2. Manejo de valores nulos en clientes
    print("Manejo de valores nulos en clientes...")
    customers['DEPARTAMENTO'] = customers['DEPARTAMENTO'].fillna('NO_ESPECIFICADO')
    customers['CIUDAD'] = customers['CIUDAD'].fillna('NO_ESPECIFICADO')
    # U para Unknown
    customers['GENERO_DIM_CLIENTE'] = customers['GENERO_DIM_CLIENTE'].fillna('U')  
    
    # Convertir fecha de nacimiento y calcular edad
    customers['FECHANACIMIENTO_DIM_CLIENTE'] = pd.to_datetime(
        customers['FECHANACIMIENTO_DIM_CLIENTE'], errors='coerce'
    )
    
    # Calcular edad
    current_year = datetime.now().year
    customers['EDAD'] = current_year - customers['FECHANACIMIENTO_DIM_CLIENTE'].dt.year
    customers['EDAD'] = customers['EDAD'].fillna(customers['EDAD'].median())
    
    # Crear grupos de edad
    def age_group(age):
        if age <= 25: return '18-25'
        elif age <= 35: return '26-35'
        elif age <= 45: return '36-45'
        elif age <= 55: return '46-55'
        else: return '55+'
    
    customers['GRUPO_EDAD'] = customers['EDAD'].apply(age_group)
    
    # 3. Ingeniería de características temporales para transacciones
    print("Ingeniería de características temporales...")
    transactions['FECHA_SOLUCION'] = pd.to_datetime(transactions['FECHA_SOLUCION'])
    transactions['MES'] = transactions['FECHA_SOLUCION'].dt.month
    transactions['DIA_SEMANA'] = transactions['FECHA_SOLUCION'].dt.day_name()
    transactions['ES_FIN_DE_SEMANA'] = transactions['FECHA_SOLUCION'].dt.dayofweek >= 5
    
    # 4. Crear variable de valor por unidad
    transactions['VALOR_POR_UNIDAD'] = transactions['VENTA_BRUTA_CON_IVA'] / transactions['UNIDADES_BRUTAS']
    
    print("Preprocesamiento completado ✓")
    return transactions, customers

# Aplicar preprocesamiento
data_transactions_clean, data_customers_clean = preprocess_data(data_transactions, data_customers)

In [ ]:
data_customers_clean.head()

In [ ]:
data_transactions_clean.tail()

# EDA

In [ ]:
# Estadísticas para transacciones
print("ESTADÍSTICAS DESCRIPTIVAS - TRANSACCIONES ")
print(data_transactions_clean[['UNIDADES_BRUTAS', 'VENTA_BRUTA_CON_IVA']].describe())

# Análisis de categorías de productos
print("\n DISTRIBUCIÓN DE CATEGORÍAS ")
category_stats = data_transactions_clean['CATEGORIA'].value_counts()
print(f"Número de categorías únicas: {len(category_stats)}")
print("Top 10 categorías:")
print(category_stats.head(10))

# Análisis temporal
data_transactions_clean['FECHA_SOLUCION'] = pd.to_datetime(data_transactions_clean['FECHA_SOLUCION'])
print(f"\nRango temporal de transacciones:")
print(f"Fecha más antigua: {data_transactions_clean['FECHA_SOLUCION'].min()}")
print(f"Fecha más reciente: {data_transactions_clean['FECHA_SOLUCION'].max()}")

In [ ]:
# Análisis del comportamiento de clientes
def customer_behavior_analysis(transactions, customers):
    print("ANÁLISIS DE COMPORTAMIENTO DE CLIENTES")
    
    # Unir datos de transacciones con información de clientes
    merged_data = transactions.merge(
        customers[['UUID_CLIENTE_CONSUMIDOR', 'GENERO_DIM_CLIENTE', 'EDAD', 'GRUPO_EDAD', 'DEPARTAMENTO']],
        on='UUID_CLIENTE_CONSUMIDOR',
        how='left'
    )
    
    # 1. Comportamiento por grupo de edad
    print("\n1. COMPORTAMIENTO POR GRUPO DE EDAD:")
    age_behavior = merged_data.groupby('GRUPO_EDAD').agg({
        'VENTA_BRUTA_CON_IVA': ['mean', 'sum', 'count'],
        'UNIDADES_BRUTAS': 'mean',
        'UUID_CLIENTE_CONSUMIDOR': 'nunique'
    }).round(2)
    print(age_behavior)
    
    # 2. Comportamiento por género
    print("\n2. COMPORTAMIENTO POR GÉNERO:")
    gender_behavior = merged_data.groupby('GENERO_DIM_CLIENTE').agg({
        'VENTA_BRUTA_CON_IVA': ['mean', 'sum'],
        'UNIDADES_BRUTAS': 'mean',
        'UUID_CLIENTE_CONSUMIDOR': 'nunique'
    }).round(2)
    print(gender_behavior)
    
    # 3. Frecuencia de compra por cliente
    print("\n3. FRECUENCIA DE COMPRA POR CLIENTE:")
    customer_frequency = transactions.groupby('UUID_CLIENTE_CONSUMIDOR').agg({
        'PEDIDO': 'count',
        'VENTA_BRUTA_CON_IVA': 'sum',
        'UNIDADES_BRUTAS': 'sum'
    }).rename(columns={'PEDIDO': 'FRECUENCIA_COMPRA'})
    
    print(f"Clientes con una sola compra: {(customer_frequency['FRECUENCIA_COMPRA'] == 1).sum()}")
    print(f"Clientes recurrentes (2+ compras): {(customer_frequency['FRECUENCIA_COMPRA'] > 1).sum()}")
    print(f"Frecuencia promedio de compra: {customer_frequency['FRECUENCIA_COMPRA'].mean():.2f}")
    
    return merged_data, customer_frequency

# Ejecutar análisis de comportamiento
merged_data, customer_frequency = customer_behavior_analysis(data_transactions_clean, data_customers_clean)

In [ ]:
# Visualizaciones más completas
def create_enhanced_visualizations(transactions, customers, customer_freq):
    fig, axes = plt.subplots(3, 2, figsize=(20, 18))
    fig.suptitle('Análisis Exploratorio de Datos', fontsize=20, fontweight='bold', y=1)
    
    # 1. Distribución de frecuencia de compra (mejorada)
    freq_counts = customer_freq['FRECUENCIA_COMPRA'].value_counts().sort_index().head(15)
    axes[0, 0].bar(freq_counts.index, freq_counts.values, color='lightseagreen', alpha=0.7)
    axes[0, 0].set_title('Distribución de Frecuencia de Compra por Cliente', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Número de Compras')
    axes[0, 0].set_ylabel('Número de Clientes')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Valor de cliente (RFM simplificado)
    top_customers = customer_freq.nlargest(10, 'VENTA_BRUTA_CON_IVA')
    customer_value = top_customers['VENTA_BRUTA_CON_IVA']

    # Crear etiquetas más informativas
    labels = []
    for i, (customer_id, row) in enumerate(top_customers.iterrows()):
        short_id = customer_id[-8:]  # Últimos 8 caracteres del UUID
        freq = row['FRECUENCIA_COMPRA']
        labels.append(f'Cliente {i+1}\n({short_id}...)\n{freq} compras')

    axes[0, 1].barh(range(len(customer_value)), customer_value.values, color='coral', alpha=0.8)
    axes[0, 1].set_yticks(range(len(customer_value)))
    axes[0, 1].set_yticklabels(labels, fontsize=9)
    axes[0, 1].set_title('Top 10 Clientes por Valor Total', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Valor Total de Compras (COP)')

    # Añadir valores en las barras
    for i, v in enumerate(customer_value.values):
        axes[0, 1].text(v + v*0.01, i, f'${v:,.0f}', 
                    va='center', fontsize=9, fontweight='bold')

    # Añadir cuadro informativo
    total_top = customer_value.sum()
    avg_value = customer_value.mean()
    axes[0, 1].text(0.02, 0.98, f'Total: ${total_top:,.0f}\nPromedio: ${avg_value:,.0f}', 
                transform=axes[0, 1].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 3. Análisis de categorías por valor
    category_value = transactions.groupby('CATEGORIA').agg({
        'VENTA_BRUTA_CON_IVA': 'sum',
        'UUID_CLIENTE_CONSUMIDOR': 'count'
    }).nlargest(10, 'VENTA_BRUTA_CON_IVA')
    
    axes[1, 0].barh(range(len(category_value)), category_value['VENTA_BRUTA_CON_IVA'].values, color='goldenrod')
    axes[1, 0].set_yticks(range(len(category_value)))
    axes[1, 0].set_yticklabels(category_value.index, fontsize=10)
    axes[1, 0].set_title('Top 10 Categorías por Valor Total de Ventas', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Valor Total de Ventas')
        
    # 5. Comportamiento por día de la semana
    weekday_sales = transactions.groupby('DIA_SEMANA')['VENTA_BRUTA_CON_IVA'].sum()
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekday_sales = weekday_sales.reindex(weekday_order)
    
    axes[1, 1].bar(range(len(weekday_sales)), weekday_sales.values, color='lightcoral')
    axes[1, 1].set_xticks(range(len(weekday_sales)))
    axes[1, 1].set_xticklabels(['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom'])
    axes[1, 1].set_title('Ventas Totales por Día de la Semana', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('Ventas Totales')
    axes[1, 1].grid(True, alpha=0.3)
    
    # 6. Distribución geográfica de ventas
    geo_sales = merged_data.groupby('DEPARTAMENTO')['VENTA_BRUTA_CON_IVA'].sum().nlargest(10)
    axes[2, 0].barh(range(len(geo_sales)), geo_sales.values, color='lightgreen')
    axes[2, 0].set_yticks(range(len(geo_sales)))
    axes[2, 0].set_yticklabels(geo_sales.index, fontsize=10)
    axes[2, 0].set_title('Top 10 Departamentos por Ventas Totales', fontsize=14, fontweight='bold')
    axes[2, 0].set_xlabel('Ventas Totales')
    
    plt.tight_layout()
    plt.show()

# Crear visualizaciones mejoradas
create_enhanced_visualizations(data_transactions_clean, data_customers_clean, customer_frequency)

In [ ]:
# 4. Distribución de valor por unidad
plt.figure(figsize=(12, 6))

# Eliminar outliers extremos para mejor visualización
Q1 = data_transactions_clean['VALOR_POR_UNIDAD'].quantile(0.01)
Q3 = data_transactions_clean['VALOR_POR_UNIDAD'].quantile(0.99)
filtered_values = data_transactions_clean[(data_transactions_clean['VALOR_POR_UNIDAD'] >= Q1) & 
                            (data_transactions_clean['VALOR_POR_UNIDAD'] <= Q3)]['VALOR_POR_UNIDAD']

plt.hist(filtered_values, bins=50, color='mediumpurple', alpha=0.7, edgecolor='black', density=True)
plt.title('Distribución de Valor por Unidad (sin outliers extremos)', fontsize=14, fontweight='bold')
plt.xlabel('Valor por Unidad (COP)')
plt.ylabel('Densidad')
plt.grid(True, alpha=0.3)

# Añadir líneas de percentiles
percentiles = [25, 50, 75, 90]
colors = ['red', 'green', 'blue', 'orange']
for p, color in zip(percentiles, colors):
    value = data_transactions_clean['VALOR_POR_UNIDAD'].quantile(p/100)
    plt.axvline(value, color=color, linestyle='--', alpha=0.8, 
                label=f'P{p}: ${value:,.0f} COP')

plt.legend()
plt.tight_layout()
plt.show()

# Preparación para el Sistema de Recomendación

In [ ]:
# Preparar datos para el modelo de recomendación
def prepare_recommendation_data(transactions):
    print("PREPARANDO DATOS PARA RECOMENDACIÓN")
    
    # Crear matriz de interacciones usuario-producto
    user_item_interactions = transactions.groupby(['UUID_CLIENTE_CONSUMIDOR', 'COD_PRODUCTO']).agg({
        # Total de unidades compradas
        'UNIDADES_BRUTAS': 'sum',  
        # Valor total gastado
        'VENTA_BRUTA_CON_IVA': 'sum',  
        # Frecuencia de compra
        'PEDIDO': 'count'  
    }).reset_index()
    
    user_item_interactions.rename(columns={'PEDIDO': 'FRECUENCIA_COMPRA'}, inplace=True)
    
    # Calcular estadísticas generales
    n_users = user_item_interactions['UUID_CLIENTE_CONSUMIDOR'].nunique()
    n_items = user_item_interactions['COD_PRODUCTO'].nunique()
    n_interactions = len(user_item_interactions)
    
    print(f"Estadísticas del Dataset:")
    print(f"- Usuarios únicos: {n_users:,}")
    print(f"- Productos únicos: {n_items:,}")
    print(f"- Interacciones totales: {n_interactions:,}")
    print(f"- Densidad de la matriz: {(n_interactions / (n_users * n_items)):.3f}")
    
    # Analizar la distribución de interacciones por usuario
    interactions_per_user = user_item_interactions.groupby('UUID_CLIENTE_CONSUMIDOR').size()
    print(f"\nDistribución de interacciones por usuario:")
    print(interactions_per_user.describe())
    
    return user_item_interactions

# Datos para recomendación
user_item_data = prepare_recommendation_data(data_transactions_clean)

In [ ]:
# Análisis para manejar el problema cold start
def cold_start_analysis(transactions, user_item_data):
    print("ANÁLISIS PARA PROBLEMA COLD START")
    
    # Productos más populares (más comprados)
    popular_products = transactions.groupby('COD_PRODUCTO').agg({
        'UUID_CLIENTE_CONSUMIDOR': 'count',
        'VENTA_BRUTA_CON_IVA': 'sum',
        'CATEGORIA': 'first'
    }).rename(columns={'UUID_CLIENTE_CONSUMIDOR': 'NUM_COMPRAS'})
    
    popular_products = popular_products.sort_values('NUM_COMPRAS', ascending=False)
    
    print("Top 10 productos más populares:")
    for i, (product_id, row) in enumerate(popular_products.head(10).iterrows(), 1):
        print(f"{i}. Producto {product_id} - {row['CATEGORIA']} - {row['NUM_COMPRAS']} compras")
    
    # Categorías más populares
    popular_categories = transactions['CATEGORIA'].value_counts().head(10)
    print(f"\nTop 10 categorías más populares:")
    for i, (category, count) in enumerate(popular_categories.items(), 1):
        print(f"{i}. {category} - {count} transacciones")
    
    return popular_products, popular_categories

# Ejecutar análisis cold start
popular_products, popular_categories = cold_start_analysis(data_transactions_clean, user_item_data)

In [ ]:
# Crear un resumen ejecutivo del análisis
def create_executive_summary(transactions, customers, user_item_data):
    print("=" * 80)
    print("RESUMEN EJECUTIVO - ANÁLISIS EXPLORATORIO DE DATOS")
    print("=" * 80)
    
    # Métricas clave
    total_transactions = len(transactions)
    total_customers = transactions['UUID_CLIENTE_CONSUMIDOR'].nunique()
    total_products = transactions['COD_PRODUCTO'].nunique()
    total_revenue = transactions['VENTA_BRUTA_CON_IVA'].sum()
    avg_transaction_value = transactions['VENTA_BRUTA_CON_IVA'].mean()
    
    print(f"\n📊 MÉTRICAS PRINCIPALES:")
    print(f"- Transacciones totales: {total_transactions:,}")
    print(f"- Clientes únicos: {total_customers:,}")
    print(f"- Productos únicos: {total_products:,}")
    print(f"- Ingresos totales: ${total_revenue:,.0f} COP")
    print(f"- Valor promedio por transacción: ${avg_transaction_value:,.0f} COP")
    
    # Comportamiento del cliente
    customer_freq = user_item_data.groupby('UUID_CLIENTE_CONSUMIDOR').size()
    repeat_customers = (customer_freq > 1).sum()
    repeat_rate = (repeat_customers / total_customers) * 100
    
    print(f"\n👥 COMPORTAMIENTO DEL CLIENTE:")
    print(f"* Tasa de clientes recurrentes: {repeat_rate:.1f}%")
    print(f"* Frecuencia promedio de compra: {customer_freq.mean():.2f}")
    print(f"* Clientes con una sola compra: {(customer_freq == 1).sum():,}")
    
    # Análisis de productos
    top_category = transactions['CATEGORIA'].value_counts().index[0]
    top_category_count = transactions['CATEGORIA'].value_counts().iloc[0]
    
    print(f"\n📦 ANÁLISIS DE PRODUCTOS:")
    print(f"+ Categoría más popular: {top_category} ({top_category_count} transacciones)")
    print(f"+ Total de categorías: {transactions['CATEGORIA'].nunique()}")
    print(f"+ Densidad user-item: {(len(user_item_data) / (total_customers * total_products)):.6f}")    
    print("\n" + "=" * 80)

# Generar resumen ejecutivo
create_executive_summary(data_transactions_clean, data_customers_clean, user_item_data)

---

<a id="seccion-5-sistema-recomendacion"></a>
# 🤖 5. Sistema de Recomendación

---

## Implementación de Algoritmos

Se implementarán 5 algoritmos de recomendación:
1. **Popularidad** (Baseline)
2. **User-Based Collaborative Filtering**
3. **Item-Based Collaborative Filtering**
4. **SVD (Matrix Factorization)**
5. **Modelo Híbrido** (Combinación de algoritmos)


In [ ]:
# Crear matriz de utilidad usuario-item
def create_user_item_matrix(transactions_df):
    print("Creando matriz usuario-item...")
    
    # Calcular frecuencia de compra y valor total por usuario-producto
    user_item_df = transactions_df.groupby(['UUID_CLIENTE_CONSUMIDOR', 'COD_PRODUCTO']).agg({
        'UNIDADES_BRUTAS': 'sum',
        'VENTA_BRUTA_CON_IVA': 'sum',
        'PEDIDO': 'count',
        'CATEGORIA': 'first'
    }).reset_index()
    
    user_item_df.rename(columns={'PEDIDO': 'FRECUENCIA_COMPRA'}, inplace=True)
    
    # Crear mapeos
    user_ids = user_item_df['UUID_CLIENTE_CONSUMIDOR'].unique()
    item_ids = user_item_df['COD_PRODUCTO'].unique()
    
    user_to_idx = {user: idx for idx, user in enumerate(user_ids)}
    idx_to_user = {idx: user for user, idx in user_to_idx.items()}
    item_to_idx = {item: idx for idx, item in enumerate(item_ids)}
    idx_to_item = {idx: item for item, idx in item_to_idx.items()}
    
    # Crear matriz sparse
    rows = [user_to_idx[user] for user in user_item_df['UUID_CLIENTE_CONSUMIDOR']]
    cols = [item_to_idx[item] for item in user_item_df['COD_PRODUCTO']]
    # Usar frecuencia como medida de preferencia
    data = user_item_df['FRECUENCIA_COMPRA'].values  
    
    utility_matrix = csr_matrix((data, (rows, cols)), 
                                shape=(len(user_ids), len(item_ids)))
    
    print(f"Matriz creada: {utility_matrix.shape[0]} usuarios, {utility_matrix.shape[1]} productos")
    print(f"Densidad: {(len(data) / (len(user_ids) * len(item_ids))):.6f}")
    
    return utility_matrix, user_to_idx, idx_to_user, item_to_idx, idx_to_item, user_item_df

# Crear matriz de utilidad
utility_matrix, user_to_idx, idx_to_user, item_to_idx, idx_to_item, user_item_df = create_user_item_matrix(data_transactions_clean)

In [ ]:
# Dividir datos en entrenamiento y prueba manteniendo la estructura de la matriz
def split_train_test(utility_matrix, test_size=0.2, random_state=42):
    print("Dividiendo datos en train/test...")
    
    # Convertir a formato COO para manipulación
    coo_matrix_data = utility_matrix.tocoo()
    
    # Obtener índices de las interacciones
    indices = list(zip(coo_matrix_data.row, coo_matrix_data.col))
    data = coo_matrix_data.data
    
    # Dividir índices
    train_indices, test_indices = train_test_split(
        range(len(indices)), test_size=test_size, random_state=random_state
    )
    
    # Crear matrices train y test
    train_rows = [indices[i][0] for i in train_indices]
    train_cols = [indices[i][1] for i in train_indices]
    train_data = [data[i] for i in train_indices]
    
    test_rows = [indices[i][0] for i in test_indices]
    test_cols = [indices[i][1] for i in test_indices]
    test_data = [data[i] for i in test_indices]
    
    train_matrix = csr_matrix((train_data, (train_rows, train_cols)), 
                                shape=utility_matrix.shape)
    test_matrix = csr_matrix((test_data, (test_rows, test_cols)), 
                            shape=utility_matrix.shape)
    
    print(f"Train: {len(train_data)} interacciones")
    print(f"Test: {len(test_data)} interacciones")
    
    return train_matrix, test_matrix, test_indices

# Dividir datos
train_matrix, test_matrix, test_indices = split_train_test(utility_matrix)

In [ ]:
# Sistema de recomendación con múltiples algoritmos
class RecommendationSystem:   
    def __init__(self):
        self.models = {}
        self.metrics = {}
        
    # Modelo basado en popularidad
    def popular_items_model(self, train_matrix, top_n=50):
        print("Entrenando modelo de popularidad...")
        
        # Calcular popularidad de items (suma de interacciones)
        item_popularity = np.array(train_matrix.sum(axis=0)).flatten()
        
        # Obtener top N items más populares
        popular_items = np.argsort(item_popularity)[::-1][:top_n]
        
        self.models['popularity'] = {
            'type': 'popularity',
            'popular_items': popular_items,
            'item_scores': item_popularity
        }
        
        return popular_items
    
    # Filtrado colaborativo basado en usuario
    def user_based_cf(self, train_matrix, min_similar_users=5):
        print("Entrenando modelo User-Based CF...")
        
        # Calcular similitud entre usuarios
        user_similarity = cosine_similarity(train_matrix)
        
        # Para cada usuario, encontrar usuarios similares
        user_predictions = {}
        for user_idx in range(train_matrix.shape[0]):
            # Obtener usuarios similares (excluyendo al mismo usuario)
            similar_users = np.argsort(user_similarity[user_idx])[::-1][1:min_similar_users+1]
            user_predictions[user_idx] = similar_users
        
        self.models['user_based'] = {
            'type': 'user_based',
            'user_similarity': user_similarity,
            'user_predictions': user_predictions
        }
        
        return user_similarity
    
    # Filtrado colaborativo basado en ítem
    def item_based_cf(self, train_matrix, min_similar_items=10):
        print("Entrenando modelo Item-Based CF...")
        
        # Calcular similitud entre ítems
        item_similarity = cosine_similarity(train_matrix.T)
        
        self.models['item_based'] = {
            'type': 'item_based',
            'item_similarity': item_similarity
        }
        
        return item_similarity
    
    # Factorización de matrices con SVD
    def matrix_factorization(self, train_matrix, n_components=50):
        print("Entrenando modelo de Factorización de Matrices...")
        
        svd = TruncatedSVD(n_components=n_components, random_state=42)
        user_factors = svd.fit_transform(train_matrix)
        item_factors = svd.components_.T
        
        self.models['svd'] = {
            'type': 'matrix_factorization',
            'model': svd,
            'user_factors': user_factors,
            'item_factors': item_factors,
            'explained_variance': svd.explained_variance_ratio_.sum()
        }
        
        print(f"Varianza explicada: {svd.explained_variance_ratio_.sum():.4f}")
        
        return user_factors, item_factors
    
    # Modelo híbrido que combina múltiples enfoques
    def hybrid_model(self, train_matrix, weights=None):
        print("Entrenando modelo híbrido...")
        
        if weights is None:
            weights = {'popularity': 0.2, 'item_based': 0.4, 'svd': 0.4}
        
        # Entrenar modelos componentes
        self.popular_items_model(train_matrix)
        self.item_based_cf(train_matrix)
        self.matrix_factorization(train_matrix)
        
        self.models['hybrid'] = {
            'type': 'hybrid',
            'weights': weights
        }
        
        return weights

# Entrenar todos los modelos
recommender = RecommendationSystem()

# Modelo de popularidad
popular_items = recommender.popular_items_model(train_matrix)

# Modelo user-based
user_similarity = recommender.user_based_cf(train_matrix)

# Modelo item-based
item_similarity = recommender.item_based_cf(train_matrix)

# Modelo de factorización de matrices
user_factors, item_factors = recommender.matrix_factorization(train_matrix, n_components=50)

# Modelo híbrido
hybrid_weights = recommender.hybrid_model(train_matrix)

In [ ]:
# Generar recomendaciones para un usuario específico
def generate_recommendations(recommender, user_id, user_to_idx, idx_to_item, 
                            item_to_idx, user_item_df, top_n=10, method='hybrid'):
    if user_id not in user_to_idx:
        return get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)
    
    user_idx = user_to_idx[user_id]
    
    if method == 'popularity':
        return _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)
    elif method == 'user_based':
        return _get_user_based_recommendations(recommender, user_idx, idx_to_item, user_item_df, top_n)
    elif method == 'item_based':
        return _get_item_based_recommendations(recommender, user_idx, train_matrix, 
                                                idx_to_item, user_item_df, top_n)
    elif method == 'svd':
        return _get_svd_recommendations(recommender, user_idx, idx_to_item, user_item_df, top_n)
    elif method == 'hybrid':
        return _get_hybrid_recommendations(recommender, user_idx, train_matrix, 
                                            idx_to_item, user_item_df, top_n)
    else:
        raise ValueError(f"Método {method} no soportado")

# Recomendaciones basadas en popularidad
def _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n):
    popular_items = recommender.models['popularity']['popular_items']
    recommendations = []
    
    for item_idx in popular_items[:top_n]:
        product_id = idx_to_item[item_idx]
        product_info = user_item_df[user_item_df['COD_PRODUCTO'] == product_id].iloc[0]
        recommendations.append({
            'COD_PRODUCTO': product_id,
            'CATEGORIA': product_info['CATEGORIA'],
            'SCORE': recommender.models['popularity']['item_scores'][item_idx],
            'METHOD': 'popularity'
        })
    
    return recommendations

# Recomendaciones user-based CF
def _get_user_based_recommendations(recommender, user_idx, idx_to_item, user_item_df, top_n):
    similar_users = recommender.models['user_based']['user_predictions'][user_idx]
    user_similarity = recommender.models['user_based']['user_similarity']
    
    # Calcular scores basados en usuarios similares
    scores = np.zeros(train_matrix.shape[1])
    for sim_user in similar_users:
        similarity_weight = user_similarity[user_idx, sim_user]
        scores += similarity_weight * train_matrix[sim_user].toarray().flatten()
    
    return _get_recommendations_from_scores(scores, user_idx, idx_to_item, user_item_df, top_n, 'user_based')

# Recomendaciones item-based CF
def _get_item_based_recommendations(recommender, user_idx, train_matrix, idx_to_item, user_item_df, top_n):
    item_similarity = recommender.models['item_based']['item_similarity']
    user_interactions = train_matrix[user_idx].toarray().flatten()
    
    # Calcular scores basados en similitud de ítems
    scores = user_interactions @ item_similarity
    
    return _get_recommendations_from_scores(scores, user_idx, idx_to_item, user_item_df, top_n, 'item_based')

# Recomendaciones con factorización de matrices
def _get_svd_recommendations(recommender, user_idx, idx_to_item, user_item_df, top_n):
    user_vector = recommender.models['svd']['user_factors'][user_idx]
    item_factors = recommender.models['svd']['item_factors']
    
    scores = user_vector @ item_factors.T
    
    return _get_recommendations_from_scores(scores, user_idx, idx_to_item, user_item_df, top_n, 'svd')

# Recomendaciones híbridas
def _get_hybrid_recommendations(recommender, user_idx, train_matrix, idx_to_item, user_item_df, top_n):
    weights = recommender.models['hybrid']['weights']
    
    # Obtener scores de cada modelo
    item_based_scores = _get_item_based_scores(recommender, user_idx, train_matrix)
    svd_scores = _get_svd_scores(recommender, user_idx)
    popularity_scores = recommender.models['popularity']['item_scores']
    
    # Normalizar scores
    item_based_scores = (item_based_scores - item_based_scores.min()) / (item_based_scores.max() - item_based_scores.min() + 1e-8)
    svd_scores = (svd_scores - svd_scores.min()) / (svd_scores.max() - svd_scores.min() + 1e-8)
    popularity_scores = (popularity_scores - popularity_scores.min()) / (popularity_scores.max() - popularity_scores.min() + 1e-8)
    
    # Combinar scores
    hybrid_scores = (weights['item_based'] * item_based_scores + 
                    weights['svd'] * svd_scores + 
                    weights['popularity'] * popularity_scores)
    
    return _get_recommendations_from_scores(hybrid_scores, user_idx, idx_to_item, user_item_df, top_n, 'hybrid')

# Obtener scores para modelo item-based
def _get_item_based_scores(recommender, user_idx, train_matrix):
    item_similarity = recommender.models['item_based']['item_similarity']
    user_interactions = train_matrix[user_idx].toarray().flatten()
    return user_interactions @ item_similarity

# Obtener scores para modelo SVD
def _get_svd_scores(recommender, user_idx):
    user_vector = recommender.models['svd']['user_factors'][user_idx]
    item_factors = recommender.models['svd']['item_factors']
    return user_vector @ item_factors.T

# Convertir scores en recomendaciones
def _get_recommendations_from_scores(scores, user_idx, idx_to_item, user_item_df, top_n, method):
    # Excluir ítems ya comprados
    purchased_items = train_matrix[user_idx].indices
    scores[purchased_items] = -np.inf
    
    # Obtener top N recomendaciones
    top_item_indices = np.argsort(scores)[::-1][:top_n]
    recommendations = []
    
    for item_idx in top_item_indices:
        if scores[item_idx] > -np.inf:
            product_id = idx_to_item[item_idx]
            product_info = user_item_df[user_item_df['COD_PRODUCTO'] == product_id].iloc[0]
            recommendations.append({
                'COD_PRODUCTO': product_id,
                'CATEGORIA': product_info['CATEGORIA'],
                'SCORE': scores[item_idx],
                'METHOD': method
            })
    
    return recommendations

# Recomendaciones populares para usuarios nuevos
def get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n):
    return _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)

In [ ]:
# Visualizar resultados de evaluación
def plot_evaluation_results(evaluation_results):
    methods = list(evaluation_results.keys())
    precision_scores = [evaluation_results[method]['precision@k'] for method in methods]
    recall_scores = [evaluation_results[method]['recall@k'] for method in methods]
    f1_scores = [evaluation_results[method]['f1_score'] for method in methods]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Precision
    bars1 = axes[0].bar(methods, precision_scores, color='skyblue', alpha=0.8)
    axes[0].set_title('Precision@10 por Modelo', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Precision')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Recall
    bars2 = axes[1].bar(methods, recall_scores, color='lightcoral', alpha=0.8)
    axes[1].set_title('Recall@10 por Modelo', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Recall')
    axes[1].tick_params(axis='x', rotation=45)
    
    # F1-Score
    bars3 = axes[2].bar(methods, f1_scores, color='lightgreen', alpha=0.8)
    axes[2].set_title('F1-Score por Modelo', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('F1-Score')
    axes[2].tick_params(axis='x', rotation=45)
    
    # Añadir valores en las barras
    for bars, ax in zip([bars1, bars2, bars3], axes):
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                    f'{height:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    plt.tight_layout()
    plt.show()

<a id="seccion-6-evaluacion"></a>
# 📈 6. Evaluación de Modelos

---

## Métricas y Comparación de Performance

Evaluaremos todos los modelos usando:
- **Precision@10**: Precisión de las recomendaciones
- **Recall@10**: Cobertura de items relevantes
- **F1-Score**: Balance entre precisión y recall


In [ ]:
def evaluate_models_fixed(recommender, test_matrix, user_to_idx, idx_to_item, user_item_df, top_n=10):
    print("Evaluando modelos (versión corregida)...")
    
    methods = ['popularity', 'item_based', 'svd', 'hybrid']
    results = {}
    
    for method in methods:
        print(f"Evaluando {method}...")
        precision_scores = []
        recall_scores = []
        
        # Evaluar en una muestra más pequeña para debug
        test_users = list(user_to_idx.keys())[:100]
        users_evaluated = 0
        
        for user_id in test_users:
            try:
                user_idx = user_to_idx[user_id]
                
                # Ítems reales en test
                actual_items = set(test_matrix[user_idx].indices)
                
                if len(actual_items) == 0:
                    continue
                    
                # CORREGIDO: Pasar top_n como entero explícitamente
                recommendations = generate_recommendations(
                    recommender, user_id, user_to_idx, idx_to_item, user_item_df, 
                    top_n=int(top_n),  # Convertir explícitamente a entero
                    method=method
                )
                
                if not recommendations:
                    continue
                    
                recommended_items = set([rec['COD_PRODUCTO'] for rec in recommendations])
                
                # Calcular métricas
                true_positives = len(actual_items.intersection(recommended_items))
                precision = true_positives / len(recommended_items)
                recall = true_positives / len(actual_items)
                
                precision_scores.append(precision)
                recall_scores.append(recall)
                users_evaluated += 1
                
            except Exception as e:
                continue
        
        if users_evaluated > 0:
            avg_precision = np.mean(precision_scores)
            avg_recall = np.mean(recall_scores)
            f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall + 1e-8)
            
            results[method] = {
                'precision@k': avg_precision,
                'recall@k': avg_recall,
                'f1_score': f1,
                'users_evaluated': users_evaluated
            }
            print(f" - Evaluado en {users_evaluated} usuarios")
            print(f" - Precision@{top_n}: {avg_precision:.4f}")
            print(f" - Recall@{top_n}: {avg_recall:.4f}")
        else:
            results[method] = {
                'precision@k': 0.0,
                'recall@k': 0.0,
                'f1_score': 0.0,
                'users_evaluated': 0
            }
            print(f"  ✗ No se pudo evaluar")
    
    return results

# También necesitamos corregir la función generate_recommendations
def generate_recommendations_fixed(recommender, user_id, user_to_idx, idx_to_item, 
                                 user_item_df, top_n=10, method='hybrid'):
    # Asegurar que top_n es entero
    top_n = int(top_n)
    
    if user_id not in user_to_idx:
        return get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)
    
    user_idx = user_to_idx[user_id]
    
    if method == 'popularity':
        return _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)
    elif method == 'user_based':
        return _get_user_based_recommendations(recommender, user_idx, idx_to_item, user_item_df, top_n)
    elif method == 'item_based':
        return _get_item_based_recommendations(recommender, user_idx, train_matrix, 
                                             idx_to_item, user_item_df, top_n)
    elif method == 'svd':
        return _get_svd_recommendations(recommender, user_idx, idx_to_item, user_item_df, top_n)
    elif method == 'hybrid':
        return _get_hybrid_recommendations(recommender, user_idx, train_matrix, 
                                         idx_to_item, user_item_df, top_n)
    else:
        raise ValueError(f"Método {method} no soportado")

# Reemplazar la función original con la corregida
generate_recommendations = generate_recommendations_fixed

# Y también corregir get_popular_recommendations
def get_popular_recommendations_fixed(recommender, idx_to_item, user_item_df, top_n=10):
    top_n = int(top_n)  # Asegurar que es entero
    return _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)

get_popular_recommendations = get_popular_recommendations_fixed

# Ahora ejecutar la evaluación corregida
print("=== EVALUACIÓN COMPLETAMENTE CORREGIDA ===")
evaluation_results = evaluate_models_fixed(recommender, test_matrix, user_to_idx, idx_to_item, user_item_df)

if evaluation_results:
    print("\n=== RESULTADOS FINALES ===")
    for method, metrics in evaluation_results.items():
        print(f"\n{method.upper()}:")
        print(f"  Precision@{10}: {metrics['precision@k']:.4f}")
        print(f"  Recall@{10}: {metrics['recall@k']:.4f}")
        print(f"  F1-Score: {metrics['f1_score']:.4f}")
        print(f"  Usuarios evaluados: {metrics['users_evaluated']}")
    
    # Visualizar resultados
    plot_evaluation_results(evaluation_results)
else:
    print("No se obtuvieron resultados de evaluación")

---

In [ ]:
# Esta sección implementa un modelo Item-Based CF mejorado con:
# - Threshold adaptativo para similitudes
# - Top-K items más similares
# - Normalización robusta
# - Fallback a popularidad para items sin similitudes
print(" APLICANDO MEJORAS ADICIONALES AL MODELO ITEM-BASED ")

# 1. MEJORA: Modelo Item-Based más robusto
def enhanced_item_based_cf(recommender, train_matrix, similarity_threshold=0.005, min_similar_items=5):
    """Item-Based CF mejorado con múltiples estrategias"""
    print("Entrenando Item-Based CF mejorado...")
    
    # Calcular similitud de coseno
    item_similarity = cosine_similarity(train_matrix.T)
    
    print(f"   - Similitud original - densidad: {np.count_nonzero(item_similarity)/item_similarity.size*100:.4f}%")
    
    # ESTRATEGIA 1: Aplicar threshold adaptativo
    item_similarity[item_similarity < similarity_threshold] = 0
    
    # ESTRATEGIA 2: Para cada ítem, mantener solo los top-K ítems más similares
    for i in range(item_similarity.shape[0]):
        row = item_similarity[i]
        # Mantener solo los top-K similares (excluyendo auto-similitud)
        top_indices = np.argsort(row)[::-1][1:min_similar_items+1]  # Excluir el propio ítem
        mask = np.zeros_like(row, dtype=bool)
        mask[top_indices] = True
        item_similarity[i] = row * mask
    
    # ESTRATEGIA 3: Añadir similitud basada en popularidad para ítems sin similitudes
    item_popularity = np.array(train_matrix.sum(axis=0)).flatten()
    popularity_similarity = item_popularity / item_popularity.max()
    
    # Combinar con similitud de coseno (peso pequeño para popularidad)
    item_similarity = item_similarity + 0.1 * popularity_similarity
    
    # ESTRATEGIA 4: Normalización robusta
    row_sums = item_similarity.sum(axis=1)
    zero_rows = row_sums == 0
    
    # Para filas con suma cero, usar popularidad normalizada
    if np.any(zero_rows):
        item_similarity[zero_rows] = popularity_similarity / popularity_similarity.sum()
        row_sums[zero_rows] = 1
    
    item_similarity = item_similarity / row_sums[:, np.newaxis]
    
    print(f"   - Después de mejoras - densidad: {np.count_nonzero(item_similarity)/item_similarity.size*100:.4f}%")
    print(f"   - Filas cero corregidas: {np.sum(zero_rows)}")
    
    # Guardar modelo mejorado
    recommender.models['item_based_enhanced'] = {
        'type': 'item_based_enhanced',
        'item_similarity': item_similarity,
        'similarity_threshold': similarity_threshold,
        'min_similar_items': min_similar_items
    }
    
    return item_similarity

# Entrenar modelo mejorado
enhanced_item_sim = enhanced_item_based_cf(recommender, train_matrix, similarity_threshold=0.005, min_similar_items=10)

In [ ]:
# Función de recomendaciones
def _get_item_based_recommendations_enhanced(recommender, user_idx, train_matrix, idx_to_item, user_item_df, top_n):
    try:
        item_similarity = recommender.models['item_based_enhanced']['item_similarity']
        user_interactions = train_matrix[user_idx].toarray().flatten()
        
        # Calcular scores
        scores = user_interactions @ item_similarity
        
        # Manejar valores problemáticos
        scores = np.nan_to_num(scores, nan=0.0, posinf=0.0, neginf=0.0)
        
        # ESTRATEGIA: Si el usuario tiene pocas interacciones, aumentar diversidad
        user_interaction_count = len(train_matrix[user_idx].indices)
        if user_interaction_count <= 2:
            # Mezclar con popularidad para usuarios con pocos datos
            popularity_scores = recommender.models['popularity']['item_scores']
            popularity_scores_norm = popularity_scores / popularity_scores.max()
            scores = 0.7 * scores + 0.3 * popularity_scores_norm
        
        # Verificar si hay scores válidos
        purchased_items = train_matrix[user_idx].indices
        scores_copy = scores.copy()
        scores_copy[purchased_items] = -np.inf
        valid_scores_mask = scores_copy > -np.inf
        
        if not np.any(valid_scores_mask) or np.max(scores_copy[valid_scores_mask]) == 0:
            # FALLBACK 1: Usar popularidad pura
            return _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)
        
        return _get_recommendations_from_scores(scores, user_idx, idx_to_item, user_item_df, top_n, 'item_based_enhanced')
    
    except Exception as e:
        print(f"      Error en item_based_enhanced: {e}")
        # FALLBACK 2: Popularidad en caso de error
        return _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)

In [ ]:
# Sistema de recomendaciones final
def generate_recommendations_final(recommender, user_id, user_to_idx, idx_to_item, 
                                    user_item_df, top_n=10, method='hybrid'):
    """Sistema final de recomendaciones con todos los modelos mejorados"""
    top_n = int(top_n)
    
    if user_id not in user_to_idx:
        return get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)
    
    user_idx = user_to_idx[user_id]
    
    method_map = {
        'popularity': lambda: _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n),
        'user_based': lambda: _get_user_based_recommendations(recommender, user_idx, idx_to_item, user_item_df, top_n),
        'item_based': lambda: _get_item_based_recommendations_enhanced(recommender, user_idx, train_matrix, idx_to_item, user_item_df, top_n),
        'svd': lambda: _get_svd_recommendations(recommender, user_idx, idx_to_item, user_item_df, top_n),
        'hybrid': lambda: _get_hybrid_recommendations_enhanced(recommender, user_idx, train_matrix, idx_to_item, user_item_df, top_n)
    }
    
    if method in method_map:
        return method_map[method]()
    else:
        raise ValueError(f"Método {method} no soportado")

# Hybrid mejorado
def _get_hybrid_recommendations_enhanced(recommender, user_idx, train_matrix, idx_to_item, user_item_df, top_n):
    """Hybrid recommendations con pesos adaptativos"""
    try:
        # Pesos base
        base_weights = {'item_based': 0.4, 'svd': 0.4, 'popularity': 0.2}
        
        # Obtener scores de cada modelo
        item_based_scores = _get_item_based_scores_enhanced(recommender, user_idx, train_matrix)
        svd_scores = _get_svd_scores(recommender, user_idx)
        popularity_scores = recommender.models['popularity']['item_scores']
        
        # ESTRATEGIA: Ajustar pesos basado en la confianza del usuario
        user_interaction_count = len(train_matrix[user_idx].indices)
        
        if user_interaction_count <= 2:
            # Usuario nuevo: dar más peso a popularidad
            weights = {'item_based': 0.3, 'svd': 0.3, 'popularity': 0.4}
        elif user_interaction_count <= 10:
            # Usuario ocasional: balance normal
            weights = base_weights
        else:
            # Usuario frecuente: más peso a modelos personalizados
            weights = {'item_based': 0.5, 'svd': 0.4, 'popularity': 0.1}
        
        # Normalizar scores
        item_based_scores_norm = _normalize_scores(item_based_scores)
        svd_scores_norm = _normalize_scores(svd_scores)
        popularity_scores_norm = _normalize_scores(popularity_scores)
        
        # Combinar scores
        hybrid_scores = (weights['item_based'] * item_based_scores_norm + 
                        weights['svd'] * svd_scores_norm + 
                        weights['popularity'] * popularity_scores_norm)
        
        return _get_recommendations_from_scores(hybrid_scores, user_idx, idx_to_item, user_item_df, top_n, 'hybrid_enhanced')
    
    except Exception as e:
        print(f"Error en hybrid mejorado: {e}")
        return _get_popular_recommendations(recommender, idx_to_item, user_item_df, top_n)

def _get_item_based_scores_enhanced(recommender, user_idx, train_matrix):
    """Obtener scores para item-based mejorado"""
    item_similarity = recommender.models['item_based_enhanced']['item_similarity']
    user_interactions = train_matrix[user_idx].toarray().flatten()
    scores = user_interactions @ item_similarity
    return np.nan_to_num(scores, nan=0.0, posinf=0.0, neginf=0.0)

def _normalize_scores(scores):
    """Normalizar scores de manera robusta"""
    scores = scores.copy()
    min_val, max_val = scores.min(), scores.max()
    
    if max_val - min_val < 1e-8:  # Evitar división por cero
        return np.ones_like(scores) * 0.5
    
    return (scores - min_val) / (max_val - min_val)

# 5. ACTUALIZAR FUNCIONES PRINCIPALES
generate_recommendations = generate_recommendations_final

In [ ]:
print("\n PROBANDO CON DIFERENTES TIPOS DE USUARIOS ")

def test_enhanced_system():
    test_users = list(user_to_idx.keys())[:10]  # Probar con 10 usuarios
    
    results = []
    
    for i, user_id in enumerate(test_users, 1):
        user_idx = user_to_idx[user_id]
        interaction_count = len(train_matrix[user_idx].indices)
        
        print(f"\nUsuario {i}: {user_id[:8]}... (Interacciones: {interaction_count})")
        
        try:
            recommendations = _get_item_based_recommendations_enhanced(
                recommender, user_idx, train_matrix, idx_to_item, user_item_df, top_n=3
            )
            
            method_used = "Item-Based"
            if not recommendations:
                method_used = "Fallback (Popularidad)"
            elif recommendations[0]['METHOD'] != 'item_based_enhanced':
                method_used = f"Fallback ({recommendations[0]['METHOD']})"
            
            print(f"  Método usado: {method_used}")
            print(f"  Recomendaciones: {len(recommendations)}")
            
            if recommendations:
                for j, rec in enumerate(recommendations[:2], 1):
                    print(f"    {j}. {rec['COD_PRODUCTO']} | {rec['CATEGORIA']} | Score: {rec['SCORE']:.4f}")
            
            results.append({
                'user_id': user_id,
                'interactions': interaction_count,
                'method_used': method_used,
                'recommendations_count': len(recommendations),
                'success': len(recommendations) > 0
            })
            
        except Exception as e:
            print(f"  Error: {e}")
            results.append({
                'user_id': user_id,
                'interactions': interaction_count,
                'method_used': 'Error',
                'recommendations_count': 0,
                'success': False
            })
    
    # Resumen de pruebas
    success_count = sum(1 for r in results if r['success'])
    fallback_count = sum(1 for r in results if 'Fallback' in r['method_used'])
    
    print(f"\n RESUMEN DE PRUEBAS ")
    print(f"Usuarios probados: {len(results)}")
    print(f"Recomendaciones exitosas: {success_count}/{len(results)} ({success_count/len(results)*100:.1f}%)")
    print(f"Usos de fallback: {fallback_count}/{len(results)} ({fallback_count/len(results)*100:.1f}%)")
    
    return results

# Ejecutar pruebas
test_results = test_enhanced_system()

In [ ]:
def evaluate_final_system(recommender, test_matrix, user_to_idx, idx_to_item, user_item_df, top_n=10, sample_size=100):
    print(f"\n EVALUACIÓN FINAL DEL SISTEMA MEJORADO (muestra: {sample_size} usuarios) ")
    
    methods = ['popularity', 'item_based', 'svd', 'hybrid']
    results = {}
    
    for method in methods:
        print(f"\nEvaluando {method.upper()}...")
        precision_scores = []
        recall_scores = []
        users_evaluated = 0
        fallback_used = 0
        
        test_users = list(user_to_idx.keys())[:sample_size]
        
        for user_id in test_users:
            try:
                user_idx = user_to_idx[user_id]
                
                # Ítems reales en test
                actual_items = set(test_matrix[user_idx].indices)
                
                if len(actual_items) == 0:
                    continue
                    
                recommendations = generate_recommendations_final(
                    recommender, user_id, user_to_idx, idx_to_item, user_item_df, 
                    top_n=int(top_n), method=method
                )
                
                if not recommendations:
                    continue
                
                # Contar si se usó fallback
                if recommendations and 'fallback' in recommendations[0]['METHOD'].lower():
                    fallback_used += 1
                    
                recommended_items = set([rec['COD_PRODUCTO'] for rec in recommendations])
                
                # Calcular métricas
                true_positives = len(actual_items.intersection(recommended_items))
                precision = true_positives / len(recommended_items)
                recall = true_positives / len(actual_items)
                
                precision_scores.append(precision)
                recall_scores.append(recall)
                users_evaluated += 1
                
            except Exception as e:
                continue
        
        if users_evaluated > 0:
            avg_precision = np.mean(precision_scores)
            avg_recall = np.mean(recall_scores)
            f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall + 1e-8)
            
            results[method] = {
                'precision@k': avg_precision,
                'recall@k': avg_recall,
                'f1_score': f1,
                'users_evaluated': users_evaluated,
                'fallback_used': fallback_used,
                'fallback_rate': fallback_used / users_evaluated if users_evaluated > 0 else 0
            }
            
            print(f"  ✅ Precision@{top_n}: {avg_precision:.4f}")
            print(f"  ✅ Recall@{top_n}: {avg_recall:.4f}")
            print(f"  ✅ F1-Score: {f1:.4f}")
            print(f"  ✅ Usuarios evaluados: {users_evaluated}")
            print(f"  🔄 Fallbacks usados: {fallback_used} ({fallback_used/users_evaluated*100:.1f}%)")
        else:
            results[method] = {
                'precision@k': 0.0,
                'recall@k': 0.0,
                'f1_score': 0.0,
                'users_evaluated': 0,
                'fallback_used': 0,
                'fallback_rate': 0
            }
            print(f"  ❌ No se pudo evaluar")
    
    return results

print("\n" + "="*70)
evaluation_final = evaluate_final_system(
    recommender, test_matrix, user_to_idx, idx_to_item, user_item_df, 
    top_n=10, sample_size=100
)

In [ ]:

# VISUALIZACIÓN FINAL DE RESULTADOS
# Comparación visual de todos los modelos evaluados


def plot_final_results(evaluation_results, title="Resultados Finales del Sistema"):
    """Visualizar resultados finales con métricas adicionales"""
    methods = list(evaluation_results.keys())
    precision_scores = [evaluation_results[method]['precision@k'] for method in methods]
    recall_scores = [evaluation_results[method]['recall@k'] for method in methods]
    f1_scores = [evaluation_results[method]['f1_score'] for method in methods]
    fallback_rates = [evaluation_results[method]['fallback_rate'] for method in methods]
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(title, fontsize=16, fontweight='bold')
    
    # Precision
    bars1 = ax1.bar(methods, precision_scores, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
    ax1.set_title('Precision@10', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Precision')
    ax1.tick_params(axis='x', rotation=45)
    
    # Recall
    bars2 = ax2.bar(methods, recall_scores, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
    ax2.set_title('Recall@10', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Recall')
    ax2.tick_params(axis='x', rotation=45)
    
    # F1-Score
    bars3 = ax3.bar(methods, f1_scores, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
    ax3.set_title('F1-Score', fontsize=14, fontweight='bold')
    ax3.set_ylabel('F1-Score')
    ax3.tick_params(axis='x', rotation=45)
    
    # Añadir valores en las barras
    for bars, ax in zip([bars1, bars2, bars3], [ax1, ax2, ax3]):
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                    f'{height:.4f}', ha='center', va='bottom', 
                    fontweight='bold', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Mostrar resultados finales
if evaluation_final:
    plot_final_results(evaluation_final, "Sistema de Recomendación - Comparación Final")
    
    print("\n" + "="*80)
    print("RESUMEN FINAL DEL SISTEMA")
    print("="*80)
    
    for method, metrics in evaluation_final.items():
        print(f"\n{method.upper():<20}:")
        print(f"  • Precision@10: {metrics['precision@k']:.4f}")
        print(f"  • Recall@10:    {metrics['recall@k']:.4f}")
        print(f"  • F1-Score:     {metrics['f1_score']:.4f}")
    
    # Identificar el mejor modelo
    best_model = max(evaluation_final.items(), key=lambda x: x[1]['f1_score'])
    print("\n" + "="*80)
    print(f"🏆 MEJOR MODELO: {best_model[0].upper()}")
    print(f"   F1-Score: {best_model[1]['f1_score']:.4f}")
    print(f"   Precision@10: {best_model[1]['precision@k']:.4f}")
    print(f"   Recall@10: {best_model[1]['recall@k']:.4f}")
    print("="*80)


---

# 🚀 PARTE 7: ESTRATEGIA DE PRODUCCIÓN Y DEPLOYMENT

A continuación se presenta el plan completo para llevar el sistema de recomendación a producción, incluyendo:
- Sistema de actualización del modelo
- Monitorización continua
- Plan de A/B testing
- Arquitectura de deployment
- Conclusiones y próximos pasos

---


<a id="seccion-7-actualizacion"></a>
# 🔄 7. Sistema de Actualización y Re-entrenamiento

---

## Estrategia de Actualización Continua

Sistema para gestionar el re-entrenamiento del modelo:
- **Incremental**: Actualizaciones diarias
- **Batch Semanal**: Re-entrenamiento completo
- **Híbrido**: Combinación de ambas estrategias
- **Triggers**: Detección automática de degradación


In [ ]:

# 7.1 SISTEMA DE ACTUALIZACIÓN Y RE-ENTRENAMIENTO


print("="*80)
print("SISTEMA DE ACTUALIZACIÓN EN TIEMPO REAL")
print("="*80)

class ModelUpdateSystem:
    """
    Sistema para gestionar actualizaciones y re-entrenamiento del modelo
    """
    
    def __init__(self, recommender, train_matrix, update_strategy='hybrid'):
        self.recommender = recommender
        self.train_matrix = train_matrix
        self.update_strategy = update_strategy
        self.update_history = []
        self.performance_history = []
        
        # Baseline performance (del modelo actual)
        self.baseline_performance = {
            'precision': 0.0034,
            'recall': 0.0230,
            'f1_score': 0.0060
        }
        
        print(f"✓ Sistema inicializado con estrategia: {update_strategy}")
        print(f"✓ Baseline F1-Score: {self.baseline_performance['f1_score']:.4f}")
    
    def define_update_strategy(self):
        """Definir estrategia de actualización"""
        
        strategies = {
            'incremental': {
                'frequency': 'Diaria',
                'scope': 'Solo nuevas interacciones últimas 24h',
                'cost': 'Bajo',
                'method': 'Actualización de matriz de similitud'
            },
            'batch_weekly': {
                'frequency': 'Semanal (Domingos 2 AM)',
                'scope': 'Todos los datos últimos 30 días',
                'cost': 'Medio',
                'method': 'Re-entrenamiento completo'
            },
            'batch_monthly': {
                'frequency': 'Mensual',
                'scope': 'Todos los datos históricos',
                'cost': 'Alto',
                'method': 'Re-entrenamiento completo + optimización'
            },
            'hybrid': {
                'frequency': 'Incremental diario + Batch semanal',
                'scope': 'Combinado',
                'cost': 'Medio',
                'method': 'Mejor de ambos mundos'
            }
        }
        
        print(f"\n📋 ESTRATEGIAS DE ACTUALIZACIÓN DISPONIBLES:\n")
        for name, config in strategies.items():
            print(f"{name.upper()}:")
            for key, value in config.items():
                print(f"  • {key.capitalize()}: {value}")
            print()
        
        return strategies[self.update_strategy]
    
    def simulate_performance_degradation(self, days=30):
        """Simular degradación del modelo con el tiempo"""
        print("\n📉 SIMULACIÓN DE DEGRADACIÓN DEL MODELO")
        
        performance_over_time = []
        base_f1 = self.baseline_performance['f1_score']
        
        for day in range(days):
            degradation = 0.005 * (day / 7)
            noise = np.random.normal(0, 0.0002)
            
            current_f1 = base_f1 * (1 - degradation) + noise
            current_f1 = max(0, current_f1)
            
            performance_over_time.append({
                'day': day,
                'f1_score': current_f1,
                'degradation_pct': degradation * 100
            })
        
        return pd.DataFrame(performance_over_time)
    
    def detect_retraining_trigger(self, current_performance, threshold=0.10):
        """Detectar si es necesario re-entrenar"""
        baseline_f1 = self.baseline_performance['f1_score']
        current_f1 = current_performance['f1_score']
        
        degradation = (baseline_f1 - current_f1) / baseline_f1
        
        triggers = {
            'performance_degradation': degradation > threshold,
            'degradation_value': degradation,
            'threshold': threshold,
            'recommendation': 'RETRAIN' if degradation > threshold else 'CONTINUE'
        }
        
        if triggers['performance_degradation']:
            print(f"\n🚨 ALERTA DE RE-ENTRENAMIENTO")
            print(f"  • Degradación detectada: {degradation:.1%}")
            print(f"  • Umbral configurado: {threshold:.1%}")
            print(f"  • Acción recomendada: RE-ENTRENAR MODELO")
        else:
            print(f"\n✅ Modelo estable")
            print(f"  • Degradación actual: {degradation:.1%}")
            print(f"  • Umbral: {threshold:.1%}")
        
        return triggers
    
    def visualize_update_strategy(self, performance_df):
        """Visualizar estrategia de actualización"""
        
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # Performance over time
        axes[0].plot(performance_df['day'], performance_df['f1_score'], 
                     'b-', linewidth=2, label='F1-Score simulado')
        axes[0].axhline(y=self.baseline_performance['f1_score'], 
                       color='g', linestyle='--', linewidth=2, label='Baseline')
        axes[0].axhline(y=self.baseline_performance['f1_score'] * 0.9, 
                       color='r', linestyle='--', linewidth=2, label='Umbral -10%')
        
        retrain_days = performance_df[performance_df['day'] % 7 == 0]
        axes[0].scatter(retrain_days['day'], retrain_days['f1_score'], 
                       color='orange', s=100, zorder=5, marker='^', 
                       label='Re-entrenamiento programado')
        
        axes[0].set_title('Evolución del Performance y Estrategia de Re-entrenamiento', 
                         fontsize=12, fontweight='bold')
        axes[0].set_xlabel('Días desde deployment')
        axes[0].set_ylabel('F1-Score')
        axes[0].legend(loc='best')
        axes[0].grid(True, alpha=0.3)
        
        # Degradación acumulada
        axes[1].fill_between(performance_df['day'], 0, 
                            performance_df['degradation_pct'], 
                            color='red', alpha=0.3)
        axes[1].plot(performance_df['day'], performance_df['degradation_pct'], 
                    'r-', linewidth=2)
        axes[1].axhline(y=10, color='orange', linestyle='--', linewidth=2, 
                       label='Umbral crítico (10%)')
        axes[1].set_title('Degradación Acumulada del Modelo', 
                         fontsize=12, fontweight='bold')
        axes[1].set_xlabel('Días desde deployment')
        axes[1].set_ylabel('Degradación (%)')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Inicializar sistema de actualización
update_system = ModelUpdateSystem(recommender, train_matrix, update_strategy='hybrid')

# Definir estrategia
strategy = update_system.define_update_strategy()

# Simular degradación
performance_df = update_system.simulate_performance_degradation(days=30)

# Detectar si necesita re-entrenamiento (día 15)
current_perf = performance_df.iloc[15]
trigger = update_system.detect_retraining_trigger(current_perf)

# Visualizar
update_system.visualize_update_strategy(performance_df)

print("\n" + "="*80)
print("✓ Sistema de actualización configurado y simulado")
print("="*80)


<a id="seccion-8-monitorizacion"></a>
# 📊 8. Monitorización Continua

---

## Dashboard de KPIs y Métricas

Sistema de monitoreo con 9 KPIs:
- **KPIs Técnicos**: Precision, Recall, F1-Score, Coverage
- **KPIs de Negocio**: CTR, Conversion Rate, AOV
- **KPIs de Calidad**: Diversity, Novelty


In [ ]:

# 7.2 SISTEMA DE MONITORIZACIÓN CONTINUA


print("\n" + "="*80)
print("SISTEMA DE MONITORIZACIÓN Y MÉTRICAS")
print("="*80)

class MonitoringSystem:
    """Sistema para monitorear el rendimiento del modelo en producción"""
    
    def __init__(self, recommender, evaluation_results):
        self.recommender = recommender
        self.baseline_metrics = evaluation_results
        self.monitoring_log = []
        self.kpis = self._define_kpis()
        
        print("✓ Sistema de monitorización inicializado")
        print(f"✓ {len(self.kpis)} KPIs configurados")
    
    def _define_kpis(self):
        """Definir KPIs técnicos y de negocio"""
        return {
            'precision@10': {'type': 'technical', 'current': 0.0034, 'target': 0.0040, 
                            'threshold_min': 0.0030, 'unit': 'score'},
            'recall@10': {'type': 'technical', 'current': 0.0230, 'target': 0.0300, 
                         'threshold_min': 0.0200, 'unit': 'score'},
            'f1_score': {'type': 'technical', 'current': 0.0060, 'target': 0.0070, 
                        'threshold_min': 0.0054, 'unit': 'score'},
            'coverage': {'type': 'technical', 'current': 0.95, 'target': 0.98, 
                        'threshold_min': 0.90, 'unit': 'percentage'},
            'ctr': {'type': 'business', 'current': 0.05, 'target': 0.0575, 
                   'threshold_min': 0.045, 'unit': 'percentage'},
            'conversion_rate': {'type': 'business', 'current': 0.02, 'target': 0.022, 
                               'threshold_min': 0.018, 'unit': 'percentage'},
            'avg_order_value': {'type': 'business', 'current': 26853, 'target': 29538, 
                               'threshold_min': 24168, 'unit': 'COP'},
            'diversity': {'type': 'quality', 'current': 5.2, 'target': 6.0, 
                         'threshold_min': 4.0, 'unit': 'categories'},
            'novelty': {'type': 'quality', 'current': 0.30, 'target': 0.35, 
                       'threshold_min': 0.20, 'unit': 'percentage'}
        }
    
    def display_kpi_dashboard(self):
        """Mostrar dashboard de KPIs"""
        print("\n" + "="*80)
        print("📊 DASHBOARD DE KPIs")
        print("="*80)
        
        for kpi_type in ['technical', 'business', 'quality']:
            type_name = {'technical': 'TÉCNICOS', 'business': 'DE NEGOCIO', 
                        'quality': 'DE CALIDAD'}[kpi_type]
            
            print(f"\n🎯 KPIs {type_name}:")
            print("-" * 80)
            
            for name, config in self.kpis.items():
                if config['type'] == kpi_type:
                    current = config['current']
                    target = config['target']
                    unit = config['unit']
                    progress = (current / target) * 100
                    
                    if current >= target:
                        status = "✅ EXCELENTE"
                    elif current >= config['threshold_min']:
                        status = "⚠️  ACEPTABLE"
                    else:
                        status = "🚨 CRÍTICO"
                    
                    if unit == 'percentage':
                        current_str = f"{current:.1%}"
                        target_str = f"{target:.1%}"
                    elif unit == 'COP':
                        current_str = f"${current:,.0f}"
                        target_str = f"${target:,.0f}"
                    else:
                        current_str = f"{current:.4f}"
                        target_str = f"{target:.4f}"
                    
                    print(f"{name.upper():<20} | Actual: {current_str:<12} | "
                          f"Target: {target_str:<12} | Progress: {progress:>5.1f}% | {status}")
    
    def calculate_quality_metrics(self, user_id):
        """Calcular métricas de calidad"""
        if user_id not in user_to_idx:
            return None
        
        recommendations = generate_recommendations_final(
            recommender, user_id, user_to_idx, idx_to_item, 
            user_item_df, top_n=10, method='hybrid'
        )
        
        if not recommendations:
            return None
        
        categories = set([rec['CATEGORIA'] for rec in recommendations])
        diversity = len(categories)
        
        rec_product_ids = [rec['COD_PRODUCTO'] for rec in recommendations]
        item_popularity = user_item_df.groupby('COD_PRODUCTO').size()
        median_popularity = item_popularity.median()
        
        novel_count = sum(
            1 for prod_id in rec_product_ids 
            if prod_id not in item_popularity.index or item_popularity[prod_id] < median_popularity
        )
        
        novelty = novel_count / len(rec_product_ids) if rec_product_ids else 0
        novelty += np.random.normal(0, 0.05)
        novelty = np.clip(novelty, 0, 1)
        
        return {'diversity': diversity, 'novelty': novelty, 
                'n_recommendations': len(recommendations)}
    
    def simulate_monitoring(self, n_users=50):
        """Simular monitoreo en múltiples usuarios"""
        print("\n📈 SIMULANDO MONITOREO EN PRODUCCIÓN...")
        
        test_users = list(user_to_idx.keys())[:n_users]
        quality_metrics = []
        
        for user_id in test_users:
            metrics = self.calculate_quality_metrics(user_id)
            if metrics:
                quality_metrics.append(metrics)
        
        if quality_metrics:
            df = pd.DataFrame(quality_metrics)
            print(f"\n✓ Métricas calculadas para {len(quality_metrics)} usuarios")
            print(f"\nEstadísticas de Calidad:")
            print(f"  • Diversidad promedio: {df['diversity'].mean():.2f} categorías")
            print(f"  • Novelty promedio: {df['novelty'].mean():.1%}")
            return df
        
        return None
    
    def visualize_monitoring(self, quality_df):
        """Visualizar métricas"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('📊 Dashboard de Monitorización en Producción', 
                     fontsize=16, fontweight='bold')
        
        # 1. KPIs Técnicos
        technical_kpis = {k: v for k, v in self.kpis.items() if v['type'] == 'technical'}
        names = list(technical_kpis.keys())
        current_values = [v['current'] for v in technical_kpis.values()]
        target_values = [v['target'] for v in technical_kpis.values()]
        
        x = np.arange(len(names))
        width = 0.35
        
        axes[0, 0].bar(x - width/2, current_values, width, label='Actual', 
                      color='skyblue', alpha=0.8)
        axes[0, 0].bar(x + width/2, target_values, width, label='Target', 
                      color='lightcoral', alpha=0.8)
        axes[0, 0].set_title('KPIs Técnicos: Actual vs Target', fontweight='bold')
        axes[0, 0].set_xticks(x)
        axes[0, 0].set_xticklabels([n.replace('_', '\n') for n in names], fontsize=9)
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. KPIs de Negocio
        business_kpis = {k: v for k, v in self.kpis.items() if v['type'] == 'business'}
        
        for i, (name, config) in enumerate(business_kpis.items()):
            progress = (config['current'] / config['target']) * 100
            color = 'green' if progress >= 100 else 'orange' if progress >= 90 else 'red'
            
            axes[0, 1].barh(i, progress, color=color, alpha=0.7)
            axes[0, 1].text(progress + 2, i, f'{progress:.1f}%', 
                          va='center', fontweight='bold')
        
        axes[0, 1].set_yticks(range(len(business_kpis)))
        axes[0, 1].set_yticklabels([n.replace('_', ' ').title() for n in business_kpis.keys()])
        axes[0, 1].axvline(x=100, color='green', linestyle='--', linewidth=2, label='Target')
        axes[0, 1].set_title('KPIs de Negocio: % de Objetivo Alcanzado', fontweight='bold')
        axes[0, 1].set_xlabel('Progreso (%)')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3, axis='x')
        
        # 3. Diversidad
        if quality_df is not None:
            axes[1, 0].hist(quality_df['diversity'], bins=10, color='purple', 
                          alpha=0.7, edgecolor='black')
            axes[1, 0].axvline(x=quality_df['diversity'].mean(), color='red', 
                             linestyle='--', linewidth=2, 
                             label=f'Media: {quality_df["diversity"].mean():.2f}')
            axes[1, 0].axvline(x=self.kpis['diversity']['target'], color='green', 
                             linestyle='--', linewidth=2, 
                             label=f'Target: {self.kpis["diversity"]["target"]:.1f}')
            axes[1, 0].set_title('Distribución de Diversidad (# Categorías)', fontweight='bold')
            axes[1, 0].set_xlabel('Número de Categorías')
            axes[1, 0].set_ylabel('Frecuencia')
            axes[1, 0].legend()
            axes[1, 0].grid(True, alpha=0.3)
        
        # 4. Novelty
        if quality_df is not None:
            novelty_values = quality_df['novelty'] * 100
            
            axes[1, 1].hist(novelty_values, bins=15, color='orange', 
                          alpha=0.7, edgecolor='black')
            
            mean_val = quality_df['novelty'].mean() * 100
            target_val = self.kpis['novelty']['target'] * 100
            
            axes[1, 1].axvline(x=mean_val, color='red', linestyle='--', linewidth=2, 
                             label=f'Media: {mean_val:.1f}%')
            axes[1, 1].axvline(x=target_val, color='green', linestyle='--', linewidth=2, 
                             label=f'Target: {target_val:.1f}%')
            
            axes[1, 1].set_title('Distribución de Novelty (% Items de Descubrimiento)', 
                               fontweight='bold')
            axes[1, 1].set_xlabel('Novelty (%)')
            axes[1, 1].set_ylabel('Frecuencia')
            axes[1, 1].set_xlim([0, 100])
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Inicializar sistema de monitorización
monitoring = MonitoringSystem(recommender, evaluation_final)
monitoring.display_kpi_dashboard()

# Simular monitoreo
quality_metrics_df = monitoring.simulate_monitoring(n_users=50)

# Visualizar
if quality_metrics_df is not None:
    monitoring.visualize_monitoring(quality_metrics_df)

print("\n" + "="*80)
print("✓ Sistema de monitorización activo")
print("="*80)


<a id="seccion-9-ab-testing"></a>
# 🧪 9. Framework de A/B Testing

---

## Diseño y Análisis de Experimentos

Framework completo para pruebas A/B:
- **Diseño de experimentos**: Hipótesis y grupos de control
- **Simulación**: Test con 1000 usuarios
- **Análisis estadístico**: Significancia con p-values
- **Visualización**: Comparación de métricas


In [ ]:

# 7.3 FRAMEWORK DE A/B TESTING


print("\n" + "="*80)
print("FRAMEWORK DE A/B TESTING")
print("="*80)

class ABTestFramework:
    """Framework para diseñar, ejecutar y analizar pruebas A/B"""
    
    def __init__(self, recommender):
        self.recommender = recommender
        self.experiments = []
        self.results = []
        print("✓ Framework de A/B Testing inicializado")
    
    def design_experiment(self):
        """Diseñar experimento A/B"""
        experiment_design = {
            'name': 'Hybrid Model vs Baseline',
            'hypothesis': {
                'h0': 'El modelo híbrido NO mejora significativamente el CTR vs baseline',
                'h1': 'El modelo híbrido mejora el CTR en al menos 15%'
            },
            'groups': {
                'control': {'name': 'Baseline (Popularidad)', 'size': 0.40, 
                           'model': 'popularity'},
                'treatment': {'name': 'Modelo Híbrido', 'size': 0.40, 
                             'model': 'hybrid'},
                'holdout': {'name': 'Sin Recomendaciones', 'size': 0.20, 
                           'model': None}
            },
            'duration': '14 días',
            'success_criteria': [
                'CTR aumenta >15% con p-value <0.05',
                'Conversion rate aumenta >10%'
            ]
        }
        
        print("\n📋 DISEÑO DEL EXPERIMENTO A/B")
        print("="*80)
        print(f"Nombre: {experiment_design['name']}")
        print(f"Duración: {experiment_design['duration']}")
        print(f"\nHipótesis:")
        print(f"  • H0: {experiment_design['hypothesis']['h0']}")
        print(f"  • H1: {experiment_design['hypothesis']['h1']}")
        
        print(f"\nGrupos:")
        for group_name, group_config in experiment_design['groups'].items():
            print(f"  • {group_config['name']} ({group_config['size']:.0%})")
        
        self.experiments.append(experiment_design)
        return experiment_design
    
    def assign_user_to_group(self, user_id, split=(0.4, 0.4, 0.2)):
        """Asignar usuario a grupo de manera determinística"""
        import hashlib
        hash_value = int(hashlib.md5(user_id.encode()).hexdigest(), 16)
        normalized = (hash_value % 10000) / 10000
        
        if normalized < split[0]:
            return 'control'
        elif normalized < split[0] + split[1]:
            return 'treatment'
        else:
            return 'holdout'
    
    def simulate_ab_test(self, n_users=1000):
        """Simular un experimento A/B"""
        print("\n🧪 SIMULANDO EXPERIMENTO A/B...")
        
        test_users = list(user_to_idx.keys())[:n_users]
        interactions = []
        
        baseline_ctr = 0.05
        baseline_conversion = 0.02
        baseline_aov = 26853
        
        for user_id in test_users:
            group = self.assign_user_to_group(user_id)
            
            if group == 'holdout':
                ctr_prob = baseline_ctr * 0.8
                conv_prob = baseline_conversion * 0.9
                aov = baseline_aov * 0.95
            elif group == 'control':
                ctr_prob = baseline_ctr
                conv_prob = baseline_conversion
                aov = baseline_aov
            else:
                ctr_prob = baseline_ctr * 1.15
                conv_prob = baseline_conversion * 1.10
                aov = baseline_aov * 1.08
            
            clicked = np.random.random() < ctr_prob
            converted = clicked and (np.random.random() < (conv_prob / ctr_prob))
            order_value = np.random.normal(aov, aov * 0.3) if converted else 0
            
            interactions.append({
                'user_id': user_id,
                'group': group,
                'clicked': clicked,
                'converted': converted,
                'order_value': max(0, order_value)
            })
        
        df = pd.DataFrame(interactions)
        
        print(f"✓ Simulación completada con {len(df)} usuarios")
        print(f"  • Control: {(df['group'] == 'control').sum()} usuarios")
        print(f"  • Treatment: {(df['group'] == 'treatment').sum()} usuarios")
        print(f"  • Holdout: {(df['group'] == 'holdout').sum()} usuarios")
        
        return df
    
    def analyze_results(self, df):
        """Analizar resultados con significancia estadística"""
        from scipy import stats
        
        print("\n📊 ANÁLISIS DE RESULTADOS")
        print("="*80)
        
        results = {}
        
        for metric in ['clicked', 'converted']:
            metric_name = 'CTR' if metric == 'clicked' else 'Conversion Rate'
            
            control_data = df[df['group'] == 'control'][metric].astype(int)
            treatment_data = df[df['group'] == 'treatment'][metric].astype(int)
            
            control_rate = control_data.mean()
            treatment_rate = treatment_data.mean()
            
            t_stat, p_value = stats.ttest_ind(control_data, treatment_data)
            lift = ((treatment_rate - control_rate) / control_rate) * 100
            is_significant = p_value < 0.05
            
            results[metric_name] = {
                'control': control_rate,
                'treatment': treatment_rate,
                'lift': lift,
                'p_value': p_value,
                'significant': is_significant
            }
            
            print(f"\n{metric_name}:")
            print(f"  • Control:   {control_rate:.2%}")
            print(f"  • Treatment: {treatment_rate:.2%}")
            print(f"  • Lift:      {lift:+.1f}%")
            print(f"  • P-value:   {p_value:.4f}")
            print(f"  • Resultado: {'✅ SIGNIFICATIVO' if is_significant else '❌ NO SIGNIFICATIVO'}")
        
        # Decisión final
        print(f"\n" + "="*80)
        print("DECISIÓN FINAL:")
        
        ctr_success = results['CTR']['significant'] and results['CTR']['lift'] > 10
        
        if ctr_success:
            decision = "✅ APROBAR DEPLOYMENT del modelo híbrido"
            print(f"  {decision}")
        else:
            decision = "❌ NO APROBAR - Continuar iterando"
            print(f"  {decision}")
        
        return results
    
    def visualize_ab_test(self, df, results):
        """Visualizar resultados del A/B test"""
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('🧪 Resultados del A/B Test: Hybrid Model vs Baseline', 
                     fontsize=16, fontweight='bold')
        
        groups = ['control', 'treatment', 'holdout']
        colors = ['#3498db', '#e74c3c', '#95a5a6']
        
        # 1. CTR
        ctr_data = [df[df['group'] == g]['clicked'].mean() for g in groups]
        bars1 = axes[0, 0].bar(groups, ctr_data, color=colors, alpha=0.8)
        axes[0, 0].set_title('Click-Through Rate por Grupo', fontweight='bold')
        axes[0, 0].set_ylabel('CTR')
        
        for bar, value in zip(bars1, ctr_data):
            axes[0, 0].text(bar.get_x() + bar.get_width()/2, value + 0.002,
                          f'{value:.2%}', ha='center', fontweight='bold')
        
        # 2. Conversion
        conv_data = [df[df['group'] == g]['converted'].mean() for g in groups]
        bars2 = axes[0, 1].bar(groups, conv_data, color=colors, alpha=0.8)
        axes[0, 1].set_title('Conversion Rate por Grupo', fontweight='bold')
        axes[0, 1].set_ylabel('Conversion Rate')
        
        for bar, value in zip(bars2, conv_data):
            axes[0, 1].text(bar.get_x() + bar.get_width()/2, value + 0.0005,
                          f'{value:.2%}', ha='center', fontweight='bold')
        
        # 3. AOV
        aov_data = [df[df['group'] == g]['order_value'].mean() for g in groups]
        bars3 = axes[1, 0].bar(groups, aov_data, color=colors, alpha=0.8)
        axes[1, 0].set_title('Average Order Value por Grupo', fontweight='bold')
        axes[1, 0].set_ylabel('AOV (COP)')
        
        for bar, value in zip(bars3, aov_data):
            axes[1, 0].text(bar.get_x() + bar.get_width()/2, value + 500,
                          f'${value:,.0f}', ha='center', fontweight='bold')
        
        # 4. Lift Summary
        lifts = {'CTR': results['CTR']['lift'], 
                'Conversion': results['Conversion Rate']['lift']}
        
        lift_values = list(lifts.values())
        lift_names = list(lifts.keys())
        lift_colors = ['green' if v > 10 else 'orange' if v > 0 else 'red' 
                      for v in lift_values]
        
        axes[1, 1].barh(lift_names, lift_values, color=lift_colors, alpha=0.8)
        axes[1, 1].axvline(x=0, color='black', linestyle='-', linewidth=1)
        axes[1, 1].axvline(x=15, color='green', linestyle='--', linewidth=2, 
                         alpha=0.5, label='Target: +15%')
        axes[1, 1].set_title('Lift: Treatment vs Control', fontweight='bold')
        axes[1, 1].set_xlabel('Lift (%)')
        axes[1, 1].legend()
        
        for i, value in enumerate(lift_values):
            axes[1, 1].text(value + 1, i, f'{value:+.1f}%', 
                          va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.show()

# Inicializar framework
ab_test = ABTestFramework(recommender)

# Diseñar experimento
experiment = ab_test.design_experiment()

# Simular experimento
test_data = ab_test.simulate_ab_test(n_users=1000)

# Analizar resultados
test_results = ab_test.analyze_results(test_data)

# Visualizar
ab_test.visualize_ab_test(test_data, test_results)

print("\n" + "="*80)
print("✓ Experimento A/B completado y analizado")
print("="*80)


<a id="seccion-10-conclusiones"></a>
# 📝 10. Conclusiones y Recomendaciones Finales

---

## Resumen Ejecutivo del Proyecto

Hallazgos principales, recomendaciones y próximos pasos:
- **Datos**: 231K transacciones, 37.5K clientes, 7.1K productos
- **Mejor Modelo**: Híbrido (F1: 0.0060, +94% vs baseline)
- **Impacto**: CTR +15%, Conversión +10%, AOV +8%
- **ROI**: Positivo en 3-4 meses


In [ ]:
# 7.5 CONCLUSIONES Y RECOMENDACIONES FINALES

print("="*80)
print("📊 RESUMEN EJECUTIVO - SISTEMA DE RECOMENDACIÓN")
print("="*80)

class ProjectSummary:
    """Generar resumen ejecutivo del proyecto"""
    
    def __init__(self):
        self.summary = {}
    
    def generate_findings(self):
        """Generar hallazgos principales"""
        
        findings = {
            'datos': [
                f"✓ Dataset: 231,000 transacciones de 37,570 clientes",
                f"✓ Catálogo: 7,134 productos en 85 categorías",
                f"✓ Densidad de matriz: 0.082% (alta sparsity)",
                f"✓ 77.4% clientes recurrentes",
                f"✓ Ticket promedio: $26,853 COP"
            ],
            'modelos': [
                f"✓ 5 algoritmos implementados y evaluados",
                f"✓ Mejor modelo: HÍBRIDO (F1: 0.0060)",
                f"✓ Mejora vs baseline: +94% en F1-Score",
                f"✓ Precision@10: 0.0034 | Recall@10: 0.0230",
                f"✓ Cold start manejado exitosamente"
            ],
            'negocio': [
                f"✓ CTR esperado: +15% vs sin recomendaciones",
                f"✓ Conversión esperada: +10%",
                f"✓ AOV esperado: +8%",
                f"✓ Costo infraestructura: $300-550 USD/mes",
                f"✓ ROI estimado: Positivo en 3-4 meses"
            ],
            'produccion': [
                f"✓ Arquitectura batch processing definida",
                f"✓ Sistema de actualización semanal",
                f"✓ 9 KPIs configurados (técnicos + negocio)",
                f"✓ Plan A/B testing diseñado (2 semanas)",
                f"✓ Rollout gradual en 5 fases"
            ]
        }
        
        print("\n🔍 HALLAZGOS PRINCIPALES\n")
        
        for category, items in findings.items():
            print(f"{category.upper()}:")
            for item in items:
                print(f"  {item}")
            print()
        
        return findings
    
    def generate_recommendations(self):
        """Generar recomendaciones"""
        
        recommendations = {
            'corto_plazo': [
                "1. EJECUTAR PILOTO (Semana 1-2)",
                "2. OPTIMIZAR MODELO (Semana 3-4)",
                "3. A/B TESTING COMPLETO (Mes 2)"
            ],
            'mediano_plazo': [
                "4. ESCALAR A PRODUCCIÓN (Mes 2-3)",
                "5. MONITOREO CONTINUO (Mes 3+)"
            ],
            'largo_plazo': [
                "6. MEJORAS AVANZADAS (Mes 4+):",
                "   - Explorar Deep Learning (Neural CF)",
                "   - Recomendaciones contextuales",
                "   - Sequence-based models"
            ]
        }
        
        print("\n💡 RECOMENDACIONES\n")
        
        for timeline, items in recommendations.items():
            timeline_name = timeline.replace('_', ' ').upper()
            print(f"{timeline_name}:")
            for item in items:
                print(f"  {item}")
            print()
        
        return recommendations
    
    def identify_risks(self):
        """Identificar riesgos"""
        
        risks = [
            {'risk': 'Baja adopción de usuarios', 'probability': 'Media', 
             'impact': 'Alto', 'mitigation': 'A/B testing + UX optimizado'},
            {'risk': 'Performance degradado', 'probability': 'Media', 
             'impact': 'Alto', 'mitigation': 'Cache + Monitoreo 24/7'},
            {'risk': 'Cold start nuevos productos', 'probability': 'Alta', 
             'impact': 'Medio', 'mitigation': 'Modelo popularidad fallback'},
            {'risk': 'Degradación del modelo', 'probability': 'Alta', 
             'impact': 'Medio', 'mitigation': 'Re-entrenamiento semanal'}
        ]
        
        print("\n⚠️  RIESGOS Y MITIGACIONES\n")
        print("-" * 100)
        print(f"{'Riesgo':<35} | {'Prob.':<8} | {'Impacto':<8} | {'Mitigación':<40}")
        print("-" * 100)
        
        for risk in risks:
            print(f"{risk['risk']:<35} | {risk['probability']:<8} | "
                  f"{risk['impact']:<8} | {risk['mitigation']:<40}")
        
        print("-" * 100)
        
        return risks
    
    def visualize_impact(self):
        """Visualizar impacto esperado"""
        
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle('📈 Impacto Esperado del Sistema de Recomendación', 
                     fontsize=16, fontweight='bold')
        
        # Gráfico 1: Mejora en métricas
        metrics = ['CTR', 'Conversion', 'AOV', 'Engagement']
        baseline = [5, 2, 26853, 100]
        with_recs = [5.75, 2.2, 29000, 110]
        improvement = [(w-b)/b*100 for b, w in zip(baseline, with_recs)]
        
        x = np.arange(len(metrics))
        width = 0.35
        
        axes[0].bar(x - width/2, baseline, width, label='Sin Recomendaciones', 
                   color='lightcoral', alpha=0.8)
        axes[0].bar(x + width/2, with_recs, width, label='Con Recomendaciones', 
                   color='lightgreen', alpha=0.8)
        
        axes[0].set_title('Métricas: Antes vs Después', fontweight='bold')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(metrics)
        axes[0].legend()
        axes[0].grid(True, alpha=0.3, axis='y')
        
        for i, imp in enumerate(improvement):
            max_val = max(baseline[i], with_recs[i])
            axes[0].text(i, max_val + max_val * 0.05,
                       f'+{imp:.1f}%', ha='center', fontweight='bold', 
                       color='green', fontsize=10)
        
        # Gráfico 2: ROI proyectado
        months = ['Mes 1', 'Mes 2', 'Mes 3', 'Mes 4', 'Mes 5', 'Mes 6']
        costs = [-500, -500, -500, -500, -500, -500]
        benefits = [0, 1000, 2500, 4000, 5500, 7000]
        roi = [b + c for b, c in zip(benefits, costs)]
        
        axes[1].plot(months, costs, 'r--', linewidth=2, marker='o', label='Costos')
        axes[1].plot(months, benefits, 'g-', linewidth=2, marker='s', label='Beneficios')
        axes[1].plot(months, roi, 'b-', linewidth=3, marker='^', label='ROI Neto')
        axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
        axes[1].fill_between(range(len(months)), 0, roi, 
                           where=[r > 0 for r in roi], 
                           color='green', alpha=0.2, label='ROI Positivo')
        
        axes[1].set_title('ROI Proyectado (USD)', fontweight='bold')
        axes[1].set_xlabel('Timeline')
        axes[1].set_ylabel('USD')
        axes[1].legend(loc='upper left')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Generar resumen ejecutivo
summary = ProjectSummary()

findings = summary.generate_findings()
recommendations = summary.generate_recommendations()
risks = summary.identify_risks()
summary.visualize_impact()

print("\n" + "="*80)
print("✅ PROYECTO COMPLETADO - SISTEMA DE RECOMENDACIÓN")
print("="*80)
print(f"\nFecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Autor: David Caleb")
print("Versión: 1.0.0")
print("\n🎯 MODELO RECOMENDADO: HÍBRIDO")
print("   • Precision@10: 0.0034")
print("   • Recall@10: 0.0230")
print("   • F1-Score: 0.0060")
print("\n📈 IMPACTO ESPERADO:")
print("   • CTR: +15%")
print("   • Conversión: +10%")
print("   • AOV: +8%")
print("\n" + "="*80)
